In [ ]:
%pip install -r requirements.txt

In [1]:
base_dir: bool = False

In [2]:
import os, sys

if not base_dir:
    os.chdir('../')
    base_dir = True

import numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn

from sklearn.metrics import accuracy_score

from utils.data_tools import inbreast, cbis_ddsm, image_processing
from utils.models import models

c:\Users\Theon\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
inbreast.download()
cbis_ddsm.download()

Path already exists for INBreast!
Path already exists for CBIS-DDSM!


In [4]:
# Get training features from cbis_ddsm and validation data from inbreast

class cbis_ddsm_data:

    x_train, x_test, y_train, y_test = data = cbis_ddsm.get_images()

class inbreast_data:

    x_train, x_test, y_train, y_test = inbreast.get_images()

    x, y = np.concat([x_train, x_test]), np.concat([y_train, y_test])

409/409


In [38]:
# Create model wrapper

M = models(cbis_ddsm_data.data)

In [ ]:
# Contruct and Train model
cnn = M.SmallCNN(epochs=3, lr=0.01, metrics=['auc', 'accuracy'])

Epoch 1/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 16s 190ms/step - accuracy: 0.7419 - auc: 0.8127 - loss: 0.5750
Epoch 2/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 176ms/step - accuracy: 0.9748 - auc: 0.9933 - loss: 0.0877
Epoch 3/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 175ms/step - accuracy: 0.9899 - auc: 0.9963 - loss: 0.0529
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.9774 - auc: 0.9991 - loss: 0.0808

            ResNet stats

            BinaryCrossentropy Loss: 0.08006858825683594
            auc: 0.9986008405685425
accuracy: 0.9825174808502197
            


In [55]:
# Validate on INBreast
y_pred = [np.round(p) for p in cnn.predict(inbreast_data.x_test, verbose=0)]

print(inbreast_accuracy := accuracy_score(inbreast_data.y_test, y_pred))

0.9878048780487805


In [56]:
# Model information

from timeit import timeit

def predict_100_images() -> None:
    cnn.predict(cbis_ddsm_data.x_test[: 10], verbose=0)

print(f"""
Validation Accuracy (INBreast): {round(inbreast_accuracy, 2) * 100}%
Number of Params: {(params := cnn.count_params())}
Memory Allocation: {round((params * 4) / (1024**2), 2)} MB
Run time (100 images): {timeit(predict_100_images, number=1):.2f} sec
""")


Validation Accuracy (INBreast): 99.0%
Number of Params: 161313
Memory Allocation: 0.62 MB
Run time (100 images): 0.14 sec

